# ETL da camada silver para camada gold

Esta célula importa todas as bibliotecas necessárias para o processo de ETL.
Aqui são carregados os pacotes para manipulação de dados (pyspark), conexão com o banco de dados PostgreSQL (psycopg), controle de mensagens de erro (sys) e tratamento de avisos (warnings).
Ela deve ser executada antes de qualquer outra célula, pois fornece as dependências básicas que serão usadas nas etapas de Extract, Transform e Load.

In [1]:
import os
import sys
import warnings
import psycopg2 
import pandas as pd
import numpy as np
from psycopg2 import sql
from dotenv import load_dotenv 

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StringType, IntegerType, DoubleType, DecimalType, TimestampType

# 1. CONFIGURAÇÕES GERAIS

como costume e para evitar possivei erro preferismo configurar o todo sparky logo no topo para ter tudo que precisamos pronto 
aproveitamos e nessa parte para Ignora avisos chatos de versão/depreciação e tambem arrega variáveis da .env


In [2]:

warnings.filterwarnings('ignore') 
load_dotenv() 

False

### 1.1 DRIVERS E CAMINHOS
nessa parte e onde feito o ajuste o caminho se necessário. O driver JDBC é vital para o Spark falar com o Postgres.

In [3]:

JAR_NAME = "postgresql-42.7.8.jar"
JAR_PATH = os.path.abspath(JAR_NAME)

if not os.path.exists(JAR_PATH):
    print(f"AVISO: Driver JDBC '{JAR_NAME}' não encontrado na pasta atual.")
    print("O Spark não conseguirá salvar no banco sem ele.")
else:
    print(f" Driver JDBC localizado: {JAR_PATH}")

 Driver JDBC localizado: /home/emivalto/workspace/BD2V/SDBD2---INEP/Transformer/postgresql-42.7.8.jar


### 1.2 INICIALIZAÇÃO DA SESSÃO SPARK
 esta e uma das fasses cruciais tambem pois e onde finalizamos sessão antiga se existir para garantir que o JAR seja carregado corretamente, e onde configuramos o Spark

In [4]:
print("\n Iniciando motor Spark...")

spark = SparkSession.builder \
    .appName("ETL_Enem_Gold_Layer") \
    .config("spark.driver.memory", "5g") \
    .config("spark.jars", JAR_PATH) \
    .config("spark.driver.extraClassPath", JAR_PATH) \
    .config("spark.executor.extraClassPath", JAR_PATH) \
    .config("spark.sql.legacy.timeParserPolicy", "CORRECTED") \
    .getOrCreate()

print(f"Spark {spark.version} ativo e pronto para análise!")



 Iniciando motor Spark...


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/24 01:55:05 WARN Utils: Your hostname, Emivalto, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/24 01:55:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
26/01/24 01:55:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark 4.1.1 ativo e pronto para análise!


### 1.3 VARIÁVEIS DE CONEXÃO


In [5]:

DB_CONFIG = {
    "host": "localhost",
    "port": "5432",
    "dbname": "dados_inep",
    "user": "admin",
    "password": "l1l2r1r2" 
}

JDBC_URL = f"jdbc:postgresql://{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"

# 1. Extract

Esta célula define as configurações de conexão com o banco de dados PostgreSQL e monta a consulta SQL que será usada para extrair os dados.
Ela cria variáveis com credenciais, monta o nome completo da tabela (schema.tabela) e gera a query SELECT * FROM silver.listings, além de preparar a connection string usada na etapa de conexão.

In [6]:

DB_SCHEMA = "silver"
TABLE_NAME = "microdados_enem"
FULL_TABLE_NAME = f"{DB_SCHEMA}.{TABLE_NAME}" 


def get_jdbc_connection_info():
    load_dotenv()
    
    url = os.getenv('DB_JDBC_URL') 
    if url:
        return url

    DB_USER = "admin"
    DB_PASS = "l1l2r1r2"
    DB_HOST = "localhost"
    DB_PORT = "5432"
    DB_NAME = "dados_inep"

    return f"jdbc:postgresql://{DB_HOST}:{DB_PORT}/{DB_NAME}", DB_USER, DB_PASS

jdbc_url, db_user, db_pass = get_jdbc_connection_info()

print(f" Preparando leitura da tabela: {FULL_TABLE_NAME}")
print(f" URL de Conexão: {jdbc_url}")

try:
    df_silver = spark.read \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", FULL_TABLE_NAME) \
        .option("user", db_user) \
        .option("password", db_pass) \
        .option("driver", "org.postgresql.Driver") \
        .load()

    print("\n Sucesso! Dados da camada Silver carregados.")
    print("Schema carregado do Banco:")
    df_silver.printSchema()

except Exception as e:
    print(f" Erro ao ler do banco: {e}")

 Preparando leitura da tabela: silver.microdados_enem
 URL de Conexão: jdbc:postgresql://localhost:5432/dados_inep

 Sucesso! Dados da camada Silver carregados.
Schema carregado do Banco:
root
 |-- nu_inscricao: long (nullable = true)
 |-- tp_faixa_etaria: integer (nullable = true)
 |-- tp_sexo: string (nullable = true)
 |-- tp_estado_civil: integer (nullable = true)
 |-- tp_cor_raca: integer (nullable = true)
 |-- tp_nacionalidade: integer (nullable = true)
 |-- tp_st_conclusao: integer (nullable = true)
 |-- tp_ano_concluiu: integer (nullable = true)
 |-- tp_escola: integer (nullable = true)
 |-- in_treineiro: integer (nullable = true)
 |-- co_municipio_prova: integer (nullable = true)
 |-- no_municipio_prova: string (nullable = true)
 |-- co_uf_prova: integer (nullable = true)
 |-- sg_uf_prova: string (nullable = true)
 |-- tp_presenca_cn: integer (nullable = true)
 |-- tp_presenca_ch: integer (nullable = true)
 |-- tp_presenca_lc: integer (nullable = true)
 |-- tp_presenca_mt: inte

# 2 CARGA DOS DADOS (Leitura JDBC via Spark) 

Esta proxima célula ira executa a extração dos dados do banco PostgreSQL.

Ela tem que estabelece a conexão usando as configurações definidas anteriormente, converte o objeto SQL em uma query, executa a consulta e carrega o resultado no DataFrame df.

Em caso de falha na conexão ou na leitura, deve ser exibido uma mensagem de erro detalhada e encerra o processo.

Em Big Data, constumasse usar o `spark.read` para criar um ponteiro distribuído para os dados, mais robusto.

afim de melhorar e ver o se aconteceu algum erro no final e feito umas captura dos erros genéricos do Java/Spark.

In [7]:
try:
    print(" Estabelecendo conexão com o Spark JDBC...")
    
    df = spark.read \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", FULL_TABLE_NAME) \
        .option("user", db_user) \
        .option("password", db_pass) \
        .option("driver", "org.postgresql.Driver") \
        .load()

    print(" Conexão estabelecida.")
    print(f" Tabela '{FULL_TABLE_NAME}' mapeada com sucesso para o Spark!")
    

except Exception as e:
   
    print(f"\n Ocorreu um erro ao conectar ou ler o banco de dados via Spark ")
    print(f"Erro detalhado: {e}")
    sys.exit(1)

 Estabelecendo conexão com o Spark JDBC...
 Conexão estabelecida.
 Tabela 'silver.microdados_enem' mapeada com sucesso para o Spark!


### 2.1 VALIDAÇÃO DA CARGA
Estas células e basicamente para validar a quantidade de dados esta vindo corretamente e exibem um resumo simples do resultado da extração, mostrando o número total de registros carregados no DataFrame df, as primeiras cinco tuplas e os tipos de cada dado.
Elas servem para confirmar visualmente que a consulta foi executada com sucesso e quantas linhas foram retornadas do banco.

In [8]:

print(" Contando registros no DataFrame...")
qtd_linhas = df.count()
print(f"Total de linhas carregadas: {qtd_linhas}")

print("\n --- Estrutura da Tabela (Schema) ---")

df.printSchema()

print("\n--- Visualização dos Dados (Amostra) ---")

df.show(5, truncate=False)



 Contando registros no DataFrame...
Total de linhas carregadas: 509954

 --- Estrutura da Tabela (Schema) ---
root
 |-- nu_inscricao: long (nullable = true)
 |-- tp_faixa_etaria: integer (nullable = true)
 |-- tp_sexo: string (nullable = true)
 |-- tp_estado_civil: integer (nullable = true)
 |-- tp_cor_raca: integer (nullable = true)
 |-- tp_nacionalidade: integer (nullable = true)
 |-- tp_st_conclusao: integer (nullable = true)
 |-- tp_ano_concluiu: integer (nullable = true)
 |-- tp_escola: integer (nullable = true)
 |-- in_treineiro: integer (nullable = true)
 |-- co_municipio_prova: integer (nullable = true)
 |-- no_municipio_prova: string (nullable = true)
 |-- co_uf_prova: integer (nullable = true)
 |-- sg_uf_prova: string (nullable = true)
 |-- tp_presenca_cn: integer (nullable = true)
 |-- tp_presenca_ch: integer (nullable = true)
 |-- tp_presenca_lc: integer (nullable = true)
 |-- tp_presenca_mt: integer (nullable = true)
 |-- co_prova_cn: integer (nullable = true)
 |-- co_prov

26/01/24 01:55:27 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+------------+------------------+------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+---------------------------------------------+---------------------------------------------+--------------------------------------------------+---------------------------------------------+---------+---------------------------------------------+---------------------------------------------+--------------------------------------------------+---------------------------------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|nu_inscricao|tp_faixa_etaria|

# 2. Transform

### RENOMEAÇÃO E REMOÇÃO DE COLUNAS 

 1. Definição do Mapa de Colunas (De -> Para)
 Adapte as chaves abaixo para as colunas do ENEM (ex: 'nu_nota_mt': 'nota_matematica')


In [9]:
print("\n Iniciando processo de transformação dos dados...")

mapa_colunas = {
    'nu_inscricao': 'NUM_INS',
    'tp_faixa_etaria': 'COD_FAI_ETA',
    'tp_sexo': 'TIP_SEX',
    'tp_estado_civil': 'COD_EST_CIV',
    'tp_cor_raca': 'COD_COR_RAC',
    'tp_nacionalidade': 'COD_NAC',
    'tp_st_conclusao': 'COD_SIT_CON',
    'tp_ano_concluiu': 'NUM_ANO_CON',
    'in_treineiro': 'IND_TRE',
    'co_municipio_prova': 'COD_MUN',
    'no_municipio_prova': 'NOM_MUN',
    'tp_presenca_cn': 'IND_PRE_NAT',
    'tp_presenca_ch': 'IND_PRE_HUM',
    'tp_presenca_lc': 'IND_PRE_LIN',
    'tp_presenca_mt': 'IND_PRE_MAT',
    'co_prova_cn': 'COD_PRV_NAT',
    'co_prova_ch': 'COD_PRV_HUM',
    'co_prova_lc': 'COD_PRV_LIN',
    'co_prova_mt': 'COD_PRV_MAT',
    'nu_nota_cn': 'VAL_NOT_NAT',
    'nu_nota_ch': 'VAL_NOT_HUM',
    'nu_nota_lc': 'VAL_NOT_LIN',
    'nu_nota_mt': 'VAL_NOT_MAT',
    'tp_lingua': 'TIP_LIN',
    'tp_status_redacao': 'COD_SIT_RED',
    'nu_nota_redacao': 'VAL_NOT_RED',
    'q001': 'COD_ESC_PAI',
    'q002': 'COD_ESC_MAE',
    'q003': 'COD_OCU_PAI',
    'q004': 'COD_OCU_MAE',
    'q005': 'QTD_PES_RES',
    'q006': 'COD_REN_FAM',
    'q007': 'COD_EMP_DOM',
    'q008': 'COD_POS_BAN',
    'q009': 'COD_POS_QUA',
    'q010': 'COD_POS_CAR',
    'q011': 'COD_POS_MOT',
    'q012': 'COD_POS_GEL',
    'q013': 'COD_POS_FRE',
    'q014': 'COD_POS_LAV',
    'q015': 'COD_POS_SEC',
    'q016': 'COD_POS_MIC',
    'q017': 'COD_POS_LOU',
    'q018': 'COD_POS_ASP',
    'q019': 'COD_POS_TEL',
    'q020': 'COD_POS_DVD',
    'q021': 'COD_POS_TVA',
    'q022': 'COD_POS_CEL',
    'q023': 'COD_POS_FIX',
    'q024': 'COD_POS_COM',
    'q025': 'IND_ACE_INT'
}


print(" Renomeando colunas...")
for coluna_antiga, coluna_nova in mapa_colunas.items():
    df = df.withColumnRenamed(coluna_antiga, coluna_nova)


cols_to_drop = ['tp_escola', 'co_uf_prova', 'sg_uf_prova','tx_gabarito_cn', 'tx_gabarito_ch', 'tx_gabarito_lc', 'tx_gabarito_mt',
                'tx_respostas_cn', 'tx_respostas_ch', 'tx_respostas_lc', 'tx_respostas_mt', 'nu_nota_comp1', 'nu_nota_comp2', 
                'nu_nota_comp3', 'nu_nota_comp4', 'nu_nota_comp5']

df = df.drop(*cols_to_drop)

print(" Colunas renomeadas e desnecessárias removidas.")
print("Novas colunas:", df.columns)

print("\n--- Visualização dos Dados (Amostra) ---")

df.show(5, truncate=False)


 Iniciando processo de transformação dos dados...
 Renomeando colunas...
 Colunas renomeadas e desnecessárias removidas.
Novas colunas: ['NUM_INS', 'COD_FAI_ETA', 'TIP_SEX', 'COD_EST_CIV', 'COD_COR_RAC', 'COD_NAC', 'COD_SIT_CON', 'NUM_ANO_CON', 'IND_TRE', 'COD_MUN', 'NOM_MUN', 'IND_PRE_NAT', 'IND_PRE_HUM', 'IND_PRE_LIN', 'IND_PRE_MAT', 'COD_PRV_NAT', 'COD_PRV_HUM', 'COD_PRV_LIN', 'COD_PRV_MAT', 'VAL_NOT_NAT', 'VAL_NOT_HUM', 'VAL_NOT_LIN', 'VAL_NOT_MAT', 'TIP_LIN', 'COD_SIT_RED', 'VAL_NOT_RED', 'COD_ESC_PAI', 'COD_ESC_MAE', 'COD_OCU_PAI', 'COD_OCU_MAE', 'QTD_PES_RES', 'COD_REN_FAM', 'COD_EMP_DOM', 'COD_POS_BAN', 'COD_POS_QUA', 'COD_POS_CAR', 'COD_POS_MOT', 'COD_POS_GEL', 'COD_POS_FRE', 'COD_POS_LAV', 'COD_POS_SEC', 'COD_POS_MIC', 'COD_POS_LOU', 'COD_POS_ASP', 'COD_POS_TEL', 'COD_POS_DVD', 'COD_POS_TVA', 'COD_POS_CEL', 'COD_POS_FIX', 'COD_POS_COM', 'IND_ACE_INT']

--- Visualização dos Dados (Amostra) ---


+------------+-----------+-------+-----------+-----------+-------+-----------+-----------+-------+-------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|NUM_INS     |COD_FAI_ETA|TIP_SEX|COD_EST_CIV|COD_COR_RAC|COD_NAC|COD_SIT_CON|NUM_ANO_CON|IND_TRE|COD_MUN|NOM_MUN     |IND_PRE_NAT|IND_PRE_HUM|IND_PRE_LIN|IND_PRE_MAT|COD_PRV_NAT|COD_PRV_HUM|COD_PRV_LIN|COD_PRV_MAT|VAL_NOT_NAT|VAL_NOT_HUM|VAL_NOT_LIN|VAL_NOT_MAT|TIP_LIN|COD_SIT_RED|VAL_NOT_RED|COD_ESC_PAI|COD_ESC_MAE|COD_OCU_PAI|COD_OCU_MAE|QTD_PES_RES|COD_REN_FAM|COD_EMP_DOM|COD_POS_BAN|COD_POS_QUA|C

### Alocação das colunas 

##### ORGANIZAÇÃO E CONVERSÃO DE TIPOS
nessa parte armazenamos as colunas nas variaveis certar para tabela dimenção e tabela fato

In [10]:


df_par = [ 'NUM_INS' , 'COD_FAI_ETA', 'TIP_SEX', 'COD_EST_CIV', 'COD_COR_RAC', 'COD_NAC', 'COD_SIT_CON', 'NUM_ANO_CON', 'IND_TRE']

df_loc = [ 'COD_MUN', 'NOM_MUN' ]

df_soc = ['COD_ESC_PAI', 'COD_ESC_MAE', 'COD_OCU_PAI', 'COD_OCU_MAE', 'QTD_PES_RES', 'COD_REN_FAM', 'COD_EMP_DOM', 'COD_POS_BAN', 'COD_POS_QUA',
          'COD_POS_CAR', 'COD_POS_MOT', 'COD_POS_GEL', 'COD_POS_FRE', 'COD_POS_LAV', 'COD_POS_SEC', 'COD_POS_MIC', 'COD_POS_LOU', 'COD_POS_ASP',
          'COD_POS_TEL', 'COD_POS_DVD', 'COD_POS_TVA', 'COD_POS_CEL', 'COD_POS_FIX', 'COD_POS_COM', 'IND_ACE_INT' ]

df_prv = ['TIP_LIN', 'COD_PRV_NAT', 'COD_PRV_HUM', 'COD_PRV_LIN', 'COD_PRV_MAT', 'COD_SIT_RED']

df_fat = ['NUM_INS','VAL_NOT_NAT', 'VAL_NOT_HUM', 'VAL_NOT_LIN', 'VAL_NOT_MAT', 'VAL_NOT_RED', 'IND_PRE_NAT', 'IND_PRE_HUM', 'IND_PRE_LIN', 
          'IND_PRE_MAT']

print(" Colunas segmentadas para Dimensões:")
print("\n Parâmetros:", df_par)
print("\n Localização:", df_loc)
print("\n Socioeconômica:", df_soc)
print("\n Prova:", df_prv)
print("\n Fato:", df_fat)


print("\nFato: Todas as outras colunas restantes.",df.columns)

 Colunas segmentadas para Dimensões:

 Parâmetros: ['NUM_INS', 'COD_FAI_ETA', 'TIP_SEX', 'COD_EST_CIV', 'COD_COR_RAC', 'COD_NAC', 'COD_SIT_CON', 'NUM_ANO_CON', 'IND_TRE']

 Localização: ['COD_MUN', 'NOM_MUN']

 Socioeconômica: ['COD_ESC_PAI', 'COD_ESC_MAE', 'COD_OCU_PAI', 'COD_OCU_MAE', 'QTD_PES_RES', 'COD_REN_FAM', 'COD_EMP_DOM', 'COD_POS_BAN', 'COD_POS_QUA', 'COD_POS_CAR', 'COD_POS_MOT', 'COD_POS_GEL', 'COD_POS_FRE', 'COD_POS_LAV', 'COD_POS_SEC', 'COD_POS_MIC', 'COD_POS_LOU', 'COD_POS_ASP', 'COD_POS_TEL', 'COD_POS_DVD', 'COD_POS_TVA', 'COD_POS_CEL', 'COD_POS_FIX', 'COD_POS_COM', 'IND_ACE_INT']

 Prova: ['TIP_LIN', 'COD_PRV_NAT', 'COD_PRV_HUM', 'COD_PRV_LIN', 'COD_PRV_MAT', 'COD_SIT_RED']

 Fato: ['NUM_INS', 'VAL_NOT_NAT', 'VAL_NOT_HUM', 'VAL_NOT_LIN', 'VAL_NOT_MAT', 'VAL_NOT_RED', 'IND_PRE_NAT', 'IND_PRE_HUM', 'IND_PRE_LIN', 'IND_PRE_MAT']

Fato: Todas as outras colunas restantes. ['NUM_INS', 'COD_FAI_ETA', 'TIP_SEX', 'COD_EST_CIV', 'COD_COR_RAC', 'COD_NAC', 'COD_SIT_CON', 'NUM_ANO_C

In [11]:
print(" Calculando volumes das Dimensões e Fato (isso pode levar alguns segundos)...")


dim_participante = df.select(*df_par).distinct()
dim_localizacao  = df.select(*df_loc).distinct()
dim_socioecon    = df.select(*df_soc).distinct()
dim_prova        = df.select(*df_prv).distinct()

fato_enem        = df.select(*df_fat) 

qtd_par = dim_participante.count()
qtd_loc = dim_localizacao.count()
qtd_soc = dim_socioecon.count()
qtd_prv = dim_prova.count()
qtd_fat = fato_enem.count()



print(" REGISTROS PREPARADOS PARA CARGA (GOLD)")

print(f" Dimensão Participante (dim_par): {qtd_par:,} registros únicos")
print(f" Dimensão Localização  (dim_loc): {qtd_loc:,} municípios únicos")
print(f" Dimensão Socioecon.   (dim_soc): {qtd_soc:,} perfis únicos")
print(f" Dimensão Prova        (dim_prv): {qtd_prv:,} variações de prova")
print(f" Tabela Fato (fato_enem):         {qtd_fat:,} registros totais")

print("--- Estrutura da Tabela Fato ---")
fato_enem.printSchema()

 Calculando volumes das Dimensões e Fato (isso pode levar alguns segundos)...


 REGISTROS PREPARADOS PARA CARGA (GOLD)
 Dimensão Participante (dim_par): 509,954 registros únicos
 Dimensão Localização  (dim_loc): 210 municípios únicos
 Dimensão Socioecon.   (dim_soc): 492,641 perfis únicos
 Dimensão Prova        (dim_prv): 495 variações de prova
 Tabela Fato (fato_enem):         509,954 registros totais
--- Estrutura da Tabela Fato ---
root
 |-- NUM_INS: long (nullable = true)
 |-- VAL_NOT_NAT: decimal(10,2) (nullable = true)
 |-- VAL_NOT_HUM: decimal(10,2) (nullable = true)
 |-- VAL_NOT_LIN: decimal(10,2) (nullable = true)
 |-- VAL_NOT_MAT: decimal(10,2) (nullable = true)
 |-- VAL_NOT_RED: decimal(10,2) (nullable = true)
 |-- IND_PRE_NAT: integer (nullable = true)
 |-- IND_PRE_HUM: integer (nullable = true)
 |-- IND_PRE_LIN: integer (nullable = true)
 |-- IND_PRE_MAT: integer (nullable = true)



# 3. Load

Esta célula configura os parâmetros de conexão com o banco de dados do Data Warehouse (gold) e valida se todas as variáveis geradas na etapa de transformação estão disponíveis na memória.
Ela garante que o ambiente esteja pronto antes de iniciar a fase de carga, evitando erros por falta de dados ou variáveis necessárias.

In [12]:
DB_SCHEMA_GOLD = "gold"

vars_to_check = [
    'dim_participante', # tabela DIM_PAR
    'dim_localizacao',  # tabela DIM_LOC
    'dim_socioecon',    # tabela DIM_SOC
    'dim_prova',        # tabela DIM_PRV
    'fato_enem',        # tabela FAT_DES
    'jdbc_url'   
]

print(f" Verificando pré-requisitos para carga no schema '{DB_SCHEMA_GOLD}'...")

for v in vars_to_check:
    if v not in globals():
        
        raise RuntimeError(f" Variável ausente: {v}. Por favor, rode a célula anterior de preparação das dimensões.")

print("Tudo certo! Todas as variáveis (DataFrames e Conexão) estão carregadas.")

 Verificando pré-requisitos para carga no schema 'gold'...
Tudo certo! Todas as variáveis (DataFrames e Conexão) estão carregadas.


In [14]:
CAMINHO_DDL_GOLD = "../gold/ddl.sql"

print(f" Lendo script DDL em: {CAMINHO_DDL_GOLD}")


try:
    with open(CAMINHO_DDL_GOLD, 'r', encoding='utf-8') as f:
        ddl_gold_script = f.read()
    print(" Arquivo lido com sucesso.")

except FileNotFoundError:
    print(f" Erro Crítico: O arquivo '{CAMINHO_DDL_GOLD}' não foi encontrado.")
    print("Certifique-se de salvar o conteúdo do DDL corrigido no arquivo.")
    sys.exit(1)

print("\n Aplicando estrutura (tabelas e chaves) no PostgreSQL...")

try:
    conn = psycopg2.connect(
        host="localhost",      
        port="5432",           
        database="dados_inep", 
        user="admin",          
        password="l1l2r1r2" 
    )
    
    with conn.cursor() as cur:
        
        cur.execute(ddl_gold_script)
        conn.commit() 
        
    conn.close()
    print(" Sucesso! Schema 'gold' e tabelas criadas/recriadas.")

except Exception as e:
    print(f" Erro ao executar o DDL no banco: {e}")
    sys.exit(1)

 Lendo script DDL em: ../gold/ddl.sql
 Arquivo lido com sucesso.

 Aplicando estrutura (tabelas e chaves) no PostgreSQL...
 Sucesso! Schema 'gold' e tabelas criadas/recriadas.


# CARGA DA DIMENSÃO PARTICIPANTE (DIM_PAR)

In [16]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import psycopg2 

print(" Preparando Dimensão Participante (DIM_PAR)...")

w = Window.orderBy("NUM_INS")

df_dim_par_final = df.select(*df_par).distinct() \
    .withColumn("PAR_SRK", F.row_number().over(w))

cols_ordem = ["PAR_SRK"] + df_par
df_dim_par_final = df_dim_par_final.select(*cols_ordem)

print(f" Inserindo {df_dim_par_final.count()} registros na tabela gold.DIM_PAR...")

try:
    df_dim_par_final.write \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "gold.DIM_PAR") \
        .option("user", db_user) \
        .option("password", db_pass) \
        .option("driver", "org.postgresql.Driver") \
        .mode("append") \
        .save()
    
    print(" Carga dos participantes concluída.")

except Exception as e:
    print(f" Erro na carga Spark: {e}")
print(" Garantindo registro 'Não Informado' (ID -1)...")

try:

    colunas_sql = ", ".join([f'"{col}"' for col in df_par])
    
    placeholders = ", ".join(["%s"] * len(df_par)) 
    valores_unknown = [-1] + [None] * len(df_par)
    sql_unknown = f"""
        INSERT INTO gold.DIM_PAR ("PAR_SRK", {colunas_sql})
        VALUES (%s, {placeholders})
        ON CONFLICT ("PAR_SRK") DO NOTHING;
    """

    conn = psycopg2.connect(
        host="localhost",      
        port="5432",           
        database="dados_inep", 
        user="admin",          
        password="l1l2r1r2" 
    )
    with conn.cursor() as cur:
        cur.execute(sql_unknown, valores_unknown)
        conn.commit()
    conn.close()
    print(" Registro 'Unknown' (-1) verificado com sucesso.")

except Exception as e:
    print(f" Erro ao inserir registro Unknown: {e}")

 Preparando Dimensão Participante (DIM_PAR)...


26/01/24 01:58:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/24 01:58:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/24 01:58:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


 Inserindo 509954 registros na tabela gold.DIM_PAR...


26/01/24 01:58:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/24 01:58:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/24 01:58:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/24 01:58:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/24 01:58:49 ERROR Executor: Exception in task 0.0 in stage 55.0 (TID 95)
java.sql.BatchUpdateException: Batch entry 0 INSERT INTO gold.DIM_PAR ("par_srk","num_ins","cod_fai_eta","tip_sex","cod_est_civ","cod_cor_rac","cod_nac","cod_sit_con","num_ano_con","ind_tre") VALUES (('1'::int4),('210051014332'::int8),('3'::int4),('F'),('1'::in

 Erro na carga Spark: An error occurred while calling o269.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 55.0 failed 1 times, most recent failure: Lost task 0.0 in stage 55.0 (TID 95) (10.255.255.254 executor driver): java.sql.BatchUpdateException: Batch entry 0 INSERT INTO gold.DIM_PAR ("par_srk","num_ins","cod_fai_eta","tip_sex","cod_est_civ","cod_cor_rac","cod_nac","cod_sit_con","num_ano_con","ind_tre") VALUES (('1'::int4),('210051014332'::int8),('3'::int4),('F'),('1'::int4),('3'::int4),('1'::int4),('2'::int4),('0'::int4),('0'::int4)) was aborted: ERROR: duplicate key value violates unique constraint "dim_par_pkey"
  Detail: Key (par_srk)=(1) already exists.  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:165)
	at org.postgresql.core.ResultHandlerDelegate.handleError(ResultHandlerDelegate.java:52)
	at org.postgresql.core.v3.QueryExecutorImpl.processRes